# Final code for Latitude and Longitude

## Using only images for training and not Region_ID

In [6]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import timm
from PIL import Image
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import gc
from collections import Counter

# --------------------- Utility Functions ---------------------
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --------------------- Paths ---------------------
TRAIN_IMG_DIR  = '/kaggle/input/iiith-images-latlong-smai/images_train/images_train/images_train'
VALID_IMG_DIR  = '/kaggle/input/iiith-images-latlong-smai/images_val/images_val'
TEST_IMG_DIR   = '/kaggle/input/iiith-images-latlong-smai/images_test/images_test'
TRAIN_LABELS   = '/kaggle/input/iiith-images-latlong-smai/cleaned_data_train.csv'
VALID_LABELS   = '/kaggle/input/iiith-images-latlong-smai/labels_val_updated.csv'
OUTPUT_CSV     = 'predictions_predict2.csv'
ANOMALIES      = [95,145,146,158,159,160,161]

# --------------------- Read DataFrames ---------------------
train_df = pd.read_csv(TRAIN_LABELS)
valid_df = pd.read_csv(VALID_LABELS)
valid_df['image_id'] = valid_df['filename'].apply(lambda x: int(x.split('_')[1].split('.')[0].lstrip('0') or '0'))
valid_df = valid_df[~valid_df['image_id'].isin(ANOMALIES)].reset_index(drop=True)

# --------------------- Debug: Check Dataset Sizes ---------------------
print(f"Original valid_df size: {len(valid_df)}")

# --------------------- Debug: Check file extensions ---------------------
def get_file_extensions(directory):
    extensions = []
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            ext = os.path.splitext(filename)[1].lower()
            extensions.append(ext)
    return Counter(extensions)

# Test if directories exist
print(f"VALID_IMG_DIR exists: {os.path.exists(VALID_IMG_DIR)}")
print(f"TEST_IMG_DIR exists: {os.path.exists(TEST_IMG_DIR)}")

# Only check extensions if directories exist
if os.path.exists(VALID_IMG_DIR):
    print(f"Valid image extensions: {get_file_extensions(VALID_IMG_DIR)}")
if os.path.exists(TEST_IMG_DIR):
    print(f"Test image extensions: {get_file_extensions(TEST_IMG_DIR)}")
    
def count_files_in_folder(folder_path):
    return sum(1 for entry in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, entry)))

# Example usage
print(f"Number of files in '{TEST_IMG_DIR}': {count_files_in_folder(TEST_IMG_DIR)}")

# --------------------- Scaling ---------------------
lat_scaler = StandardScaler().fit(train_df[['latitude']])
long_scaler = StandardScaler().fit(train_df[['longitude']])
for df in [train_df, valid_df]:
    df['scaled_lat'] = lat_scaler.transform(df[['latitude']])
    df['scaled_lon'] = long_scaler.transform(df[['longitude']])

# --------------------- Transforms ---------------------
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(0.3,0.3,0.3,0.05),
    transforms.RandomAffine(20, translate=(0.15,0.15), scale=(0.85,1.15)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    transforms.RandomErasing(0.2, scale=(0.02,0.15), ratio=(0.3,3.3))
])
val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# --------------------- Dataset ---------------------
class GeoDataset(Dataset):
    def __init__(self, img_dir, df, transform=None, is_test=False):
        self.img_dir = img_dir
        self.df = df.copy()
        self.transform = transform
        self.is_test = is_test
        self.df = self.df[self.df['filename'].apply(lambda fn: os.path.exists(os.path.join(img_dir, fn)))].reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.img_dir, row['filename'])).convert('RGB')
        x = self.transform(img)
        if self.is_test:
            return {'image': x, 'image_id': int(row['filename'].split('_')[1].split('.')[0].lstrip('0') or '0')}
        return {
            'image': x,
            'scaled_lat': torch.tensor(row['scaled_lat'], dtype=torch.float32),
            'scaled_lon': torch.tensor(row['scaled_lon'], dtype=torch.float32),
            'latitude': torch.tensor(row['latitude'], dtype=torch.float32),
            'longitude': torch.tensor(row['longitude'], dtype=torch.float32)
        }

# --------------------- DataLoaders ---------------------
batch_size = 16
train_loader = DataLoader(GeoDataset(TRAIN_IMG_DIR, train_df, train_transform), batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(GeoDataset(VALID_IMG_DIR, valid_df, val_test_transform), 32, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(GeoDataset(TEST_IMG_DIR, valid_df, val_test_transform, is_test=True), 32, shuffle=False, num_workers=4, pin_memory=True)

# --------------------- Model ---------------------
class SwinGeoWithoutRegion(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=0, global_pool='avg')
        feat = self.backbone.num_features
        self.fuse = nn.Sequential(
            nn.Linear(feat, 1024), nn.LayerNorm(1024), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(1024, 512), nn.LayerNorm(512), nn.GELU(), nn.Dropout(0.2)
        )
        self.lat_head = nn.Sequential(nn.Linear(512, 128), nn.GELU(), nn.Linear(128, 1))
        self.lon_head = nn.Sequential(nn.Linear(512, 128), nn.GELU(), nn.Linear(128, 1))
    def forward(self, x):
        feats = self.backbone(x)
        h = self.fuse(feats)
        return self.lat_head(h).squeeze(-1), self.lon_head(h).squeeze(-1)

model = SwinGeoWithoutRegion().to(device)

# --------------------- Loss, Optimizer & Scheduler ---------------------
class GeoLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    def forward(self, pred_lat, pred_lon, true_lat, true_lon):
        return self.mse(pred_lat, true_lat) + self.mse(pred_lon, true_lon)

criterion = GeoLoss()
params_backbone, params_new = [], []
for name, param in model.named_parameters():
    if 'backbone' in name:
        params_backbone.append(param)
    else:
        params_new.append(param)

optimizer = optim.AdamW([
    {'params': params_backbone, 'lr': 1e-5},
    {'params': params_new,      'lr': 2e-4}
], weight_decay=1e-2)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=[3e-5,5e-4], steps_per_epoch=len(train_loader), epochs=30)

# --------------------- Training & Evaluation ---------------------
def train_eval():
    best_mse = float('inf')
    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    history = {'train_loss': [], 'train_lat_loss': [], 'train_lon_loss': [],
               'val_loss': [], 'val_lat_loss': [], 'val_lon_loss': [],
               'val_unscaled_mse': [], 'lr': []}

    for epoch in range(1, 31):
        model.train()
        running_loss = running_lat = running_lon = 0.0
        count = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} Train")
        for batch in pbar:
            imgs = batch['image'].to(device)
            lat_s = batch['scaled_lat'].to(device)
            lon_s = batch['scaled_lon'].to(device)
            
            # Forward pass without region ID
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                p_lat, p_lon = model(imgs)
                loss = criterion(p_lat, p_lon, lat_s, lon_s)
                lat_loss = nn.MSELoss()(p_lat, lat_s)
                lon_loss = nn.MSELoss()(p_lon, lon_s)
            
            optimizer.zero_grad()
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            scheduler.step()

            bs = imgs.size(0)
            running_loss += loss.item() * bs
            running_lat += lat_loss.item() * bs
            running_lon += lon_loss.item() * bs
            count += bs
            lr = optimizer.param_groups[0]['lr']
            pbar.set_postfix({'loss': loss.item(), 'lat': lat_loss.item(), 'lon': lon_loss.item(), 'lr': lr})

        epoch_loss = running_loss / count
        epoch_lat = running_lat / count
        epoch_lon = running_lon / count
        history['train_loss'].append(epoch_loss)
        history['train_lat_loss'].append(epoch_lat)
        history['train_lon_loss'].append(epoch_lon)
        history['lr'].append(lr)
        print(f"Epoch {epoch} TRAIN -> Loss: {epoch_loss:.6f}, Lat: {epoch_lat:.6f}, Lon: {epoch_lon:.6f}, LR: {lr:.6e}")

        model.eval()
        v_loss = v_lat = v_lon = 0.0
        v_count = 0
        all_preds, all_true = [], []
        vbar = tqdm(valid_loader, desc=f"Epoch {epoch} Val")
        with torch.no_grad():
            for batch in vbar:
                imgs = batch['image'].to(device)
                
                # Forward pass without region ID
                p_lat, p_lon = model(imgs)
                lat_s = batch['scaled_lat'].to(device)
                lon_s = batch['scaled_lon'].to(device)

                loss = criterion(p_lat, p_lon, lat_s, lon_s)
                lat_loss = nn.MSELoss()(p_lat, lat_s)
                lon_loss = nn.MSELoss()(p_lon, lon_s)

                ulat = lat_scaler.inverse_transform(p_lat.cpu().numpy().reshape(-1,1)).flatten()
                ulon = long_scaler.inverse_transform(p_lon.cpu().numpy().reshape(-1,1)).flatten()
                all_preds.append(np.vstack([ulat, ulon]).T)
                tr_lat = batch['latitude'].numpy()
                tr_lon = batch['longitude'].numpy()
                all_true.append(np.vstack([tr_lat, tr_lon]).T)

                bs = imgs.size(0)
                v_loss += loss.item() * bs
                v_lat += lat_loss.item() * bs
                v_lon += lon_loss.item() * bs
                v_count += bs
                vbar.set_postfix({'v_loss': loss.item(), 'v_lat': lat_loss.item(), 'v_lon': lon_loss.item()})

        val_loss = v_loss / v_count
        val_lat = v_lat / v_count
        val_lon = v_lon / v_count
        preds = np.concatenate(all_preds)
        true  = np.concatenate(all_true)
        unscaled_mse = ((preds - true)**2).mean()

        history['val_loss'].append(val_loss)
        history['val_lat_loss'].append(val_lat)
        history['val_lon_loss'].append(val_lon)
        history['val_unscaled_mse'].append(unscaled_mse)

        print(f"Epoch {epoch} VAL   -> Loss: {val_loss:.6f}, Lat: {val_lat:.6f}, Lon: {val_lon:.6f}, Unscaled MSE: {unscaled_mse:.6f}")

        if unscaled_mse < best_mse:
            best_mse = unscaled_mse
            torch.save(model.state_dict(), 'best_geo2.pth')

        torch.cuda.empty_cache(); gc.collect()

    model.load_state_dict(torch.load('best_geo2.pth'))
    return history

def generate_csv(best_mse):
    rows = []
    model.eval()
    
    # Create a list to track processed IDs to avoid duplicates
    processed_ids = set()
    
    print("\n--- Starting CSV Generation ---")
    print(f"Valid loader dataset size: {len(valid_loader.dataset)}")
    
    # Process validation data with more detailed tracking
    valid_count = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Processing validation data"):
            imgs = batch['image'].to(device)
            
            # Forward pass without region ID
            p_lat, p_lon = model(imgs)
            ulat = lat_scaler.inverse_transform(p_lat.cpu().numpy().reshape(-1,1)).flatten()
            ulon = long_scaler.inverse_transform(p_lon.cpu().numpy().reshape(-1,1)).flatten()
            
            # Process each image in the batch
            batch_size = imgs.size(0)
            for i in range(batch_size):
                if i + valid_count >= len(valid_loader.dataset):
                    continue
                    
                # Get the image ID
                image_id = valid_loader.dataset.df.iloc[valid_count + i]['image_id']
                
                # Check if we've already processed this ID
                if image_id in processed_ids:
                    print(f"Warning: Duplicate ID {image_id} in validation data")
                    continue
                    
                processed_ids.add(image_id)
                rows.append({
                    'id': image_id, 
                    'Latitude': ulat[i], 
                    'Longitude': ulon[i]
                })
            
            valid_count += batch_size
    
    print(f"Processed {valid_count} validation images, added {len(processed_ids)} unique IDs")
    
    # Process test data (IDs 369–737)
    test_start_id = 369
    test_end_id = 737
    test_count = 0
    
    # Check if test directory exists and list files
    if os.path.exists(TEST_IMG_DIR):
        test_files = sorted(os.listdir(TEST_IMG_DIR))
        print(f"Found {len(test_files)} files in test directory")
        
        # Create a list to hold test file information
        test_file_list = []
        
        # Check if files have 'img_' prefix
        if test_files and any(fn.startswith('img_') for fn in test_files):
            print("Detected 'img_' prefix in test files")
            
            # Map test files to test IDs sequentially
            for i, filename in enumerate(test_files):
                if i < (test_end_id - test_start_id + 1):  # Ensure we don't exceed the test ID range
                    test_id = test_start_id + i
                    test_file_list.append({
                        'filename': filename,
                        'image_id': test_id
                    })
            
            print(f"Created mapping for {len(test_file_list)} test files")
            if len(test_file_list) > 0:
                print(f"Sample mapping: {test_file_list[:3]}")
    else:
        print(f"Test directory {TEST_IMG_DIR} not found")
        test_file_list = []
    
    # Process test images if we have any
    if test_file_list:
        # Define a dataset for test images
        class TestImageDataset(Dataset):
            def __init__(self, img_dir, file_list, transform):
                self.img_dir = img_dir
                self.file_list = file_list
                self.transform = transform
                
            def __len__(self):
                return len(self.file_list)
                
            def __getitem__(self, idx):
                file_info = self.file_list[idx]
                img_path = os.path.join(self.img_dir, file_info['filename'])
                img = Image.open(img_path).convert('RGB')
                return {
                    'image': self.transform(img),
                    'image_id': file_info['image_id']
                }
        
        # Create test dataset and dataloader
        test_dataset = TestImageDataset(TEST_IMG_DIR, test_file_list, val_test_transform)
        test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
        
        print(f"Created test dataloader with {len(test_dataset)} images")
        
        # Process test data
        with torch.no_grad():
            for batch in tqdm(test_dataloader, desc="Processing test data"):
                imgs = batch['image'].to(device)
                image_ids = batch['image_id'].tolist()  # Convert tensor to list
                
                # Forward pass without region ID
                p_lat, p_lon = model(imgs)
                ulat = lat_scaler.inverse_transform(p_lat.cpu().numpy().reshape(-1,1)).flatten()
                ulon = long_scaler.inverse_transform(p_lon.cpu().numpy().reshape(-1,1)).flatten()
                
                for i in range(len(image_ids)):
                    img_id = image_ids[i]
                    
                    # Check if we've already processed this ID
                    if img_id in processed_ids:
                        print(f"Warning: Duplicate ID {img_id} in test data")
                        continue
                        
                    processed_ids.add(img_id)
                    rows.append({
                        'id': img_id, 
                        'Latitude': ulat[i], 
                        'Longitude': ulon[i]
                    })
                    test_count += 1
        
        print(f"Processed {test_count} test images")
    
    # Handle any missing test IDs (fill with zeros or nearest neighbor)
    missing_test_ids = set(range(test_start_id, test_end_id + 1)) - processed_ids
    if missing_test_ids:
        print(f"Warning: {len(missing_test_ids)} test IDs missing. Adding placeholders.")
        for img_id in missing_test_ids:
            rows.append({
                'id': img_id,
                'Latitude': 0.0,  # Use a default or interpolate from nearest neighbors
                'Longitude': 0.0
            })
    
    # Save CSV
    result_df = pd.DataFrame(rows).sort_values('id')
    result_df.to_csv(OUTPUT_CSV, index=False)
    
    print(f"\nCSV saved with {len(result_df)} rows:")
    print(f"- {len(processed_ids.intersection(set(range(0, test_start_id))))} validation entries")
    print(f"- {len(processed_ids.intersection(set(range(test_start_id, test_end_id + 1))))} test entries")
    print(f"- {len(missing_test_ids)} placeholder entries for missing test IDs")
    
    # Report best MSE
    if isinstance(best_mse, dict):
        best_mse_value = best_mse.get('val_unscaled_mse', [-1])[-1] if 'val_unscaled_mse' in best_mse else "N/A"
        print(f"Best MSE: {best_mse_value}")
    else:
        print(f"Best MSE: {best_mse:.6f}")
            
# --------------------- Main ---------------------
best_mse = train_eval()
generate_csv(best_mse)

print("Training complete.")

Using device: cuda
Original valid_df size: 362
VALID_IMG_DIR exists: True
TEST_IMG_DIR exists: True
Valid image extensions: Counter({'.jpg': 246, '.jpeg': 96, '.png': 27})
Test image extensions: Counter({'.jpg': 241, '.jpeg': 101, '.png': 27})
Number of files in '/kaggle/input/iiith-images-latlong-smai/images_test/images_test': 369


/tmp/ipykernel_31/3891777002.py:170: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
Epoch 1 Train:   0%|          | 0/405 [00:00<?, ?it/s]/tmp/ipykernel_31/3891777002.py:186: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Epoch 1 Train: 100%|██████████| 405/405 [01:45<00:00,  3.82it/s, loss=0.00707, lat=0.000206, lon=0.00686, lr=2.07e-6]


Epoch 1 TRAIN -> Loss: 1.476800, Lat: 0.775467, Lon: 0.701334, LR: 2.068898e-06


Epoch 1 Val: 100%|██████████| 12/12 [00:05<00:00,  2.27it/s, v_loss=0.285, v_lat=0.173, v_lon=0.113]


Epoch 1 VAL   -> Loss: 0.910843, Lat: 0.495232, Lon: 0.415611, Unscaled MSE: 503451.937500


...

Epoch 30 VAL   -> Loss: 0.050950, Lat: 0.029835, Lon: 0.021115, Unscaled MSE: 27547.646484

--- Starting CSV Generation ---
Valid loader dataset size: 362


Processing validation data: 100%|██████████| 12/12 [00:05<00:00,  2.30it/s]


Processed 362 validation images, added 362 unique IDs
Found 369 files in test directory
Detected 'img_' prefix in test files
Created mapping for 369 test files
Sample mapping: [{'filename': 'img_0000.jpg', 'image_id': 369}, {'filename': 'img_0001.jpg', 'image_id': 370}, {'filename': 'img_0002.jpg', 'image_id': 371}]
Created test dataloader with 369 images


Processing test data: 100%|██████████| 12/12 [00:05<00:00,  2.26it/s]

Processed 369 test images

CSV saved with 731 rows:
- 362 validation entries
- 369 test entries
- 0 placeholder entries for missing test IDs
Best MSE: 27547.646484375
Training complete.


# Fine-tuning for more epochs

In [8]:
# Improved retraining code with better LR scheduling
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau

# Load the previously trained model
model.load_state_dict(torch.load('/kaggle/working/best_geo2.pth'))

# Reset optimizer with lower learning rates for fine-tuning
optimizer = optim.AdamW([
    {'params': params_backbone, 'lr': 5e-6},  # Lower LR for backbone
    {'params': params_new, 'lr': 1e-4}        # Lower LR for new layers
], weight_decay=1e-3)  # Slightly reduced weight decay

# Better LR scheduler - Cosine annealing with warm restarts
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-7)
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
# Alternative: ReduceLROnPlateau (uncomment to use instead)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-7, verbose=True)

# Number of additional training epochs
additional_epochs = 20
best_mse = float('inf')

# Training loop
for epoch in range(1, additional_epochs+1):
    model.train()
    running_loss = running_lat = running_lon = 0.0
    count = 0
    
    for batch in tqdm(train_loader, desc=f"Retrain Epoch {epoch}"):
        imgs = batch['image'].to(device)
        lat_s = batch['scaled_lat'].to(device)
        lon_s = batch['scaled_lon'].to(device)
        
        # Forward pass
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            p_lat, p_lon = model(imgs)
            loss = criterion(p_lat, p_lon, lat_s, lon_s)
        
        optimizer.zero_grad()
        if torch.cuda.is_available():
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        bs = imgs.size(0)
        running_loss += loss.item() * bs
        count += bs
    
    epoch_loss = running_loss / count
    print(f"Retrain Epoch {epoch} Train Loss: {epoch_loss:.6f}")
    
    # Evaluate
    model.eval()
    v_loss = 0.0
    v_count = 0
    all_preds, all_true = [], []
    
    with torch.no_grad():
        for batch in valid_loader:
            imgs = batch['image'].to(device)
            lat_s = batch['scaled_lat'].to(device)
            lon_s = batch['scaled_lon'].to(device)
            
            p_lat, p_lon = model(imgs)
            loss = criterion(p_lat, p_lon, lat_s, lon_s)
            
            # Calculate unscaled predictions for MSE
            ulat = lat_scaler.inverse_transform(p_lat.cpu().numpy().reshape(-1,1)).flatten()
            ulon = long_scaler.inverse_transform(p_lon.cpu().numpy().reshape(-1,1)).flatten()
            all_preds.append(np.vstack([ulat, ulon]).T)
            tr_lat = batch['latitude'].numpy()
            tr_lon = batch['longitude'].numpy()
            all_true.append(np.vstack([tr_lat, tr_lon]).T)
            
            bs = imgs.size(0)
            v_loss += loss.item() * bs
            v_count += bs
    
    val_loss = v_loss / v_count
    preds = np.concatenate(all_preds)
    true = np.concatenate(all_true)
    unscaled_mse = ((preds - true)**2).mean()
    
    print(f"Retrain Epoch {epoch} Val Loss: {val_loss:.6f}, Unscaled MSE: {unscaled_mse:.6f}")
    
    # Update scheduler (use this for ReduceLROnPlateau)
    # scheduler.step(unscaled_mse)
    
    # Update scheduler (use this for CosineAnnealingWarmRestarts)
    scheduler.step()
    
    # Save best model
    if unscaled_mse < best_mse:
        best_mse = unscaled_mse
        torch.save(model.state_dict(), 'best_retrained_geo2.pth')
        print(f"New best model saved with MSE: {best_mse:.6f}")
    
    torch.cuda.empty_cache()
    gc.collect()

# Load best retrained model and generate predictions
model.load_state_dict(torch.load('best_retrained_geo2.pth'))
generate_csv(best_mse)

print(f"Retraining complete. Best MSE: {best_mse:.6f}")

/tmp/ipykernel_31/2636405055.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/best_geo2.pth'))
/tmp/ipykernel_31/2636405

Retrain Epoch 1 Train Loss: 0.028745


Retrain Epoch 1 Val Loss: 0.067705, Unscaled MSE: 37737.519531
New best model saved with MSE: 37737.519531


Retrain Epoch 2:   0%|          | 0/405 [00:00<?, ?it/s]/tmp/ipykernel_31/2636405055.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Retrain Epoch 2: 100%|██████████| 405/405 [01:45<00:00,  3.84it/s]

Retrain Epoch 2 Train Loss: 0.026634


...


--- Starting CSV Generation ---
Valid loader dataset size: 362


Processing validation data: 100%|██████████| 12/12 [00:05<00:00,  2.30it/s]


Processed 362 validation images, added 362 unique IDs
Found 369 files in test directory
Detected 'img_' prefix in test files
Created mapping for 369 test files
Sample mapping: [{'filename': 'img_0000.jpg', 'image_id': 369}, {'filename': 'img_0001.jpg', 'image_id': 370}, {'filename': 'img_0002.jpg', 'image_id': 371}]
Created test dataloader with 369 images


Processing test data: 100%|██████████| 12/12 [00:05<00:00,  2.24it/s]

Processed 369 test images

CSV saved with 731 rows:
- 362 validation entries
- 369 test entries
- 0 placeholder entries for missing test IDs
Best MSE: 22757.236328
Retraining complete. Best MSE: 22757.236328


# Final code :- angle using only images and not Region_ID

In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
import timm
import math
import random
from torch.cuda.amp import GradScaler, autocast

# -----------------------
# Paths and Globals
# -----------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 96     # increased batch size
EPOCHS = 60        # more epochs
BASE_LR = 5e-4      # higher learning rate
WEIGHT_DECAY = 1e-4 # added weight decay
SEED = 42           # added seed for reproducibility

TRAIN_CSV = '/kaggle/input/iiith-images-latlong-smai/cleaned_data_train.csv'
TRAIN_IMG_DIR = '/kaggle/input/iiith-images-latlong-smai/images_train/images_train/images_train/'
VAL_CSV = '/kaggle/input/iiith-images-latlong-smai/labels_val_updated.csv'
VAL_IMG_DIR = '/kaggle/input/iiith-images-latlong-smai/images_val/images_val/'
MODEL_PATH = '/kaggle/working/efficientnet_angle_regressor.pt'

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SEED)

# -----------------------
# Dataset - REMOVED Region_ID dependency
# -----------------------
class CampusDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, is_val=False):
        df = pd.read_csv(csv_file)
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_val = is_val
        if is_val:
            self.df['idx'] = self.df.index
        # Convert angles to radians for smoother learning
        self.df['angle_rad'] = self.df['angle'] * (math.pi / 180.0)
        # Create sin and cos components for circular regression - ensure float32
        self.df['sin_angle'] = np.sin(self.df['angle_rad']).astype(np.float32)
        self.df['cos_angle'] = np.cos(self.df['angle_rad']).astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        angle = float(row['angle'])
        sin_angle = float(row['sin_angle'])
        cos_angle = float(row['cos_angle'])
        
        if self.is_val:
            return img, torch.tensor(angle, dtype=torch.float32), torch.tensor(sin_angle, dtype=torch.float32), torch.tensor(cos_angle, dtype=torch.float32), torch.tensor(int(row['idx']), dtype=torch.long)
        return img, torch.tensor(angle, dtype=torch.float32), torch.tensor(sin_angle, dtype=torch.float32), torch.tensor(cos_angle, dtype=torch.float32)

# -----------------------
# Augmentations - Same as original
# -----------------------
train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2)
])

val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# -----------------------
# Model: EfficientNet WITHOUT Region Conditioning
# -----------------------
class AngleRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        # Use EfficientNet B0 as backbone
        self.backbone = timm.create_model('efficientnet_b0', pretrained=True, features_only=True)
        
        # Extract feature dimensions from the backbone
        dummy_input = torch.zeros(1, 3, 224, 224)
        features = self.backbone(dummy_input)
        feature_dim = features[-1].shape[1]  # Last feature map channels
        
        # Global average pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Feature processing
        self.features = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.2)
        )
        
        # Sin/Cos prediction - circular regression approach
        self.head = nn.Linear(256, 2)
        
        # Enable gradient checkpointing if available
        if hasattr(self.backbone, 'gradient_checkpointing_enable'):
            self.backbone.gradient_checkpointing_enable()

    def forward(self, x):
        features = self.backbone(x)
        x = self.global_pool(features[-1]).squeeze(-1).squeeze(-1)
        
        # Process features
        x = self.features(x)
        
        # Predict sin and cos components
        sin_cos = self.head(x)
        sin_pred, cos_pred = sin_cos.split(1, dim=1)
        
        # Normalize the output to ensure it falls on the unit circle - ensure float32
        norm = torch.sqrt(sin_pred**2 + cos_pred**2) + 1e-8
        sin_norm = sin_pred / norm
        cos_norm = cos_pred / norm
        
        # Convert to angle in degrees
        angle = torch.atan2(sin_norm, cos_norm) * (180.0 / torch.tensor(math.pi, dtype=torch.float32, device=sin_pred.device))
        # Ensure angle is in [0, 360)
        angle = (angle + 360) % 360
        
        return angle.squeeze(1), sin_norm.squeeze(1), cos_norm.squeeze(1)

# -----------------------
# Loss Functions - Same as original
# -----------------------
def circle_loss(sin_pred, cos_pred, sin_true, cos_true):
    # MSE loss between the normalized sin and cos components
    return nn.MSELoss()(sin_pred, sin_true) + nn.MSELoss()(cos_pred, cos_true)

def maae_loss(pred, true):
    diff = torch.abs(pred - true)
    return torch.mean(torch.min(diff, 360 - diff))

# -----------------------
# Mixup Implementation - Modified to remove region_id dependency
# -----------------------
def mixup_data(x, y_angle, y_sin, y_cos, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    lam = float(lam)  # Ensure lam is a float32 compatible value
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    
    # We mix angles in the sin/cos space to handle the circular nature
    mixed_sin = lam * y_sin + (1 - lam) * y_sin[index]
    mixed_cos = lam * y_cos + (1 - lam) * y_cos[index]
    
    # Reconstruct angle from sin/cos - ensure torch.float32
    mixed_angle = torch.atan2(mixed_sin, mixed_cos) * (180.0 / math.pi)
    mixed_angle = (mixed_angle + 360) % 360
    
    return mixed_x, mixed_angle, mixed_sin, mixed_cos, index, lam

# -----------------------
# DataLoaders
# -----------------------
train_ds = CampusDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=train_tf)
val_ds = CampusDataset(VAL_CSV, VAL_IMG_DIR, transform=val_tf, is_val=True)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# -----------------------
# Model, Optimizer, AMP, Scheduler
# -----------------------
model = AngleRegressor().to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
scaler = GradScaler()

# OneCycleLR for faster convergence
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=BASE_LR,
    steps_per_epoch=len(train_loader),
    epochs=EPOCHS,
    pct_start=0.1,
    div_factor=10.0,
    final_div_factor=1000.0
)

# -----------------------
# EMA Model (Exponential Moving Average for better stability) - Same as original
# -----------------------
class EMA():
    def __init__(self, model, decay=0.999):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}
        
    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()
                
    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = self.decay * self.shadow[name] + (1.0 - self.decay) * param.data
                self.shadow[name] = new_average.clone()
                
    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]
                
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

# Initialize EMA
ema = EMA(model, decay=0.998)
ema.register()

# -----------------------
# Training & Validation - Modified to remove region_id dependency
# -----------------------
best_maae = float('inf')
val_maae_history = []
train_loss_history = []

print(f"Training on {DEVICE} with {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"Model: EfficientNet B0 Image-Only")
print(f"Batch Size: {BATCH_SIZE}, Epochs: {EPOCHS}, Base LR: {BASE_LR}")

for epoch in range(1, EPOCHS+1):
    # Train
    model.train()
    train_loss = 0
    train_angle_loss = 0
    train_circle_loss = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch}/{EPOCHS} [Train]')
    for imgs, angles, sin_angles, cos_angles in progress_bar:
        imgs = imgs.to(DEVICE)
        angles = angles.to(DEVICE)
        sin_angles = sin_angles.to(DEVICE)
        cos_angles = cos_angles.to(DEVICE)

        # Apply mixup with 50% probability
        if random.random() < 0.5:
            imgs, mixed_angles, mixed_sin, mixed_cos, _, _ = mixup_data(
                imgs, angles, sin_angles, cos_angles
            )
            sin_angles, cos_angles = mixed_sin, mixed_cos
            angles = mixed_angles

        optimizer.zero_grad()
        
        with autocast():
            pred_angles, pred_sin, pred_cos = model(imgs)
            
            # Combined loss: angle MAAE + sin-cos circle loss
            angle_loss = maae_loss(pred_angles, angles)
            circ_loss = circle_loss(pred_sin, pred_cos, sin_angles, cos_angles)
            loss = angle_loss * 0.5 + circ_loss * 0.5
            
        scaler.scale(loss).backward()
        
        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        # Update EMA model
        ema.update()
        
        train_loss += loss.item() * imgs.size(0)
        train_angle_loss += angle_loss.item() * imgs.size(0)
        train_circle_loss += circ_loss.item() * imgs.size(0)
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f"{loss.item():.4f}", 
            'angle_loss': f"{angle_loss.item():.4f}",
            'lr': f"{optimizer.param_groups[0]['lr']:.6f}"
        })
    
    train_loss /= len(train_loader.dataset)
    train_angle_loss /= len(train_loader.dataset)
    train_circle_loss /= len(train_loader.dataset)
    train_loss_history.append(train_loss)
    
    # Validate with EMA model
    ema.apply_shadow()
    model.eval()
    val_loss = 0
    all_preds, all_trues, all_indices = [], [], []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc=f'Epoch {epoch}/{EPOCHS} [Val]')
        for imgs, angles, sin_angles, cos_angles, indices in progress_bar:
            imgs = imgs.to(DEVICE)
            angles = angles.to(DEVICE)
            sin_angles = sin_angles.to(DEVICE)
            cos_angles = cos_angles.to(DEVICE)
            
            pred_angles, pred_sin, pred_cos = model(imgs)
            angle_loss = maae_loss(pred_angles, angles)
            val_loss += angle_loss.item() * imgs.size(0)
            
            all_preds.append(pred_angles.cpu().numpy())
            all_trues.append(angles.cpu().numpy())
            all_indices.append(indices.numpy())
            
            # Update progress bar
            progress_bar.set_postfix({'val_loss': f"{angle_loss.item():.4f}"})
    
    # Restore original model
    ema.restore()
            
    val_loss /= len(val_loader.dataset)
    preds = np.concatenate(all_preds)
    trues = np.concatenate(all_trues)
    indices = np.concatenate(all_indices)
    
    # Calculate MAAE (Mean Absolute Angular Error)
    val_maae = np.mean(np.minimum(np.abs(preds-trues), 360-np.abs(preds-trues)))
    val_maae_history.append(val_maae)
    
    print(f"Epoch {epoch}/{EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f} (Angle: {train_angle_loss:.4f}, Circle: {train_circle_loss:.4f})")
    print(f"  Val MAAE: {val_maae:.4f}")
    
    # Save best model
    if val_maae < best_maae:
        best_maae = val_maae
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'ema_shadow': ema.shadow,
            'optimizer_state_dict': optimizer.state_dict(),
            'val_maae': val_maae,
            'best_maae': best_maae,
        }, MODEL_PATH)
        print(f"  Saved best model (MAAE: {best_maae:.4f})")

print(f"\nTraining completed!")
print(f"Best validation MAAE: {best_maae:.4f}")

# -----------------------
# Plot training history - Same as original
# -----------------------
try:
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(1, EPOCHS+1), train_loss_history, label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, EPOCHS+1), val_maae_history, label='Val MAAE')
    plt.axhline(y=best_maae, color='r', linestyle='--', label=f'Best MAAE: {best_maae:.4f}')
    plt.title('Validation MAAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAAE (degrees)')
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.close()
except:
    print("Could not generate training history plot.")
    

/tmp/ipykernel_31/1015802815.py:210: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Training on cuda with Tesla T4
Model: EfficientNet B0 Image-Only
Batch Size: 96, Epochs: 60, Base LR: 0.0005


Epoch 1/60 [Train]:   0%|          | 0/68 [00:00<?, ?it/s]/tmp/ipykernel_31/1015802815.py:298: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoch 1/60 [Val]: 100%|██████████| 4/4 [00:01<00:00,  2.61it/s, val_loss=96.3957]


Epoch 1/60
  Train Loss: 45.1741 (Angle: 88.4477, Circle: 1.9006)
  Val MAAE: 90.2269
  Saved best model (MAAE: 90.2269)


Epoch 2/60 [Val]: 100%|██████████| 4/4 [00:01<00:00,  2.57it/s, val_loss=91.1004]


...

Epoch 58/60
  Train Loss: 5.9642 (Angle: 11.8162, Circle: 0.1123)
  Val MAAE: 29.0653


Epoch 59/60 [Val]: 100%|██████████| 4/4 [00:01<00:00,  2.28it/s, val_loss=22.3818]


Epoch 59/60
  Train Loss: 5.1313 (Angle: 10.1734, Circle: 0.0891)
  Val MAAE: 28.6878


Epoch 60/60 [Val]: 100%|██████████| 4/4 [00:01<00:00,  2.62it/s, val_loss=22.7880]


Epoch 60/60
  Train Loss: 5.7940 (Angle: 11.4813, Circle: 0.1067)
  Val MAAE: 27.8772
  Saved best model (MAAE: 27.8772)

Training completed!
Best validation MAAE: 27.8772


## Predicting and saving in csv

In [16]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

# ---- Paths ----
VAL_CSV       = '/kaggle/input/iiith-images-latlong-smai/labels_val_updated.csv'
VAL_IMG_DIR   = '/kaggle/input/iiith-images-latlong-smai/images_val/images_val'
TEST_IMG_DIR  = '/kaggle/input/iiith-images-latlong-smai/images_test/images_test'
ANGLE_MODEL_PATH = '/kaggle/working/efficientnet_angle_regressor.pt'
OUTPUT_CSV    = '/kaggle/working/final_predictions.csv'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ---- Transforms ----
val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ---- TestImageDataset (same) ----
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.files = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.img_dir, self.files[idx])).convert('RGB')
        if self.transform: img = self.transform(img)
        return img, idx

# ---- Load Angle Regressor ----
angle_model = AngleRegressor()
ckpt = torch.load(ANGLE_MODEL_PATH, map_location=DEVICE)
angle_model.load_state_dict(ckpt['model_state_dict'])
angle_model.to(DEVICE).eval()
# apply EMA if present
if 'ema_shadow' in ckpt:
    for n, p in angle_model.named_parameters():
        if n in ckpt['ema_shadow']:
            p.data = ckpt['ema_shadow'][n].clone()

# ---- Dataloaders ----
val_ds  = CampusDataset(VAL_CSV, VAL_IMG_DIR, transform=val_tf, is_val=True)
test_ds = TestImageDataset(TEST_IMG_DIR, transform=val_tf)
val_loader  = DataLoader(val_ds,  batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4)

# ---- Predict Function ----
def predict_angles(angle_model, dataloader):
    preds = np.zeros(len(dataloader.dataset), dtype=np.float32)
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            # Unpack img batch and index batch regardless of val vs test
            if len(batch) == 2:
                imgs, indices = batch               # test_loader: (img, idx)
            else:
                imgs, *_, indices = batch           # val_loader: (img, angle, sin, cos, idx)
            
            imgs = imgs.to(DEVICE)

            # Predict angles
            ang, _, _ = angle_model(imgs)

            # Store into preds array
            for i, idx in enumerate(indices):
                preds[idx] = ang[i].item()
    return preds

# ---- Run Prediction & Save ----
val_preds  = predict_angles(angle_model, val_loader)
test_preds = predict_angles(angle_model, test_loader)

# combine with correct indexing
all_preds = np.concatenate([val_preds, test_preds], axis=0)
df = pd.DataFrame({'id': np.arange(len(all_preds)), 'angle': all_preds})
df.to_csv(OUTPUT_CSV, index=False)

print(f" Saved predictions to {OUTPUT_CSV}")
print(f" Val (0–{len(val_preds)-1}): mean={val_preds.mean():.2f}, min={val_preds.min():.2f}, max={val_preds.max():.2f}")
print(f" Test ({len(val_preds)}–{len(all_preds)-1}): mean={test_preds.mean():.2f}, min={test_preds.min():.2f}, max={test_preds.max():.2f}")


/tmp/ipykernel_31/1579367137.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ANGLE_MODEL_PATH, map_location=DEVICE)
Predicting: 100%|██████████| 12/12

 Saved predictions to /kaggle/working/final_predictions.csv
 Val (0–368): mean=181.66, min=0.07, max=359.70
 Test (369–737): mean=187.17, min=2.69, max=359.60
